In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
tf.__version__

'2.7.0'

In [3]:
train_set = pd.read_csv('train.csv')
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [58]:
def encode_gender(g):
    return 0 if g == 'male' else 1

def decode_gender(g):
    return 'male' if g== 0 else 'female'

def encode_ticket(ticket):
    t = ticket.split()

    if t[-1].isalpha():
        return 0
    return int(t[-1])

def encode_cabin(cabin):
    if not cabin: # handle NaN
        return 0,0
    
    # todo : parse string part

    # todo : parse numeric part
    return 0, 0

def encode_embarked(embarked_class):
    if embarked_class == 'S': return 1
    if embarked_class == 'C': return 2
    if embarked_class == 'Q': return 3
    return 0

def serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender):
    ticket = encode_ticket(ticket)
    embarked = encode_embarked(embarked)
    gender = encode_gender(gender)
    cabin_type, cabin_id = encode_cabin(cabin)
    
    # FIXME : not gonna use Name for now
    if label:
        feature = {
            'pass_id' : _int64_feature(pass_id),
            'label' : _int64_feature(label),
            'pclass' : _int64_feature(pclass),
            'sibsp' : _int64_feature(sibsp),
            'parch' : _int64_feature(parch),
            'ticket' : _int64_feature(ticket),
            'fare' : _float_feature(fare),
            'embarked' : _int64_feature(embarked),
            'gender' : _int64_feature(gender),
            'cabin_type' : _int64_feature(cabin_type),
            'cabin_id' : _int64_feature(cabin_id),
        }
    else:
        feature = {
            'pass_id' : _int64_feature(pass_id),
            'pclass' : _int64_feature(pclass),
            'sibsp' : _int64_feature(sibsp),
            'parch' : _int64_feature(parch),
            'ticket' : _int64_feature(ticket),
            'fare' : _float_feature(fare),
            'embarked' : _int64_feature(embarked),
            'gender' : _int64_feature(gender),
            'cabin_type' : _int64_feature(cabin_type),
            'cabin_id' : _int64_feature(cabin_id),
        }

    # Create a Features message using tf.train.Example

    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature)
    )
    return example_proto.SerializeToString()
    

In [45]:
train_set['Ticket']

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [46]:
x = 0
for _, row in train_set.iterrows():
    x+=1
    pass_id, label, pclass, name, age,sibsp,patch, ticket, fare, cabin, embarked = row.PassengerId, row.Survived, row.Pclass, row.Name, row.Age, row.SibSp, row.Parch, row.Ticket, row.Fare, row.Cabin, row.Embarked
    print(row)
    if x == 2:
        break

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                               22.0
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object
PassengerId                                                    2
Survived                                                       1
Pclass                                                         1
Name           Cumings, Mrs. John Bradley (Florence Briggs Th...
Sex                                                       female
Age                                                         38.0
SibSp                                                          1
Parch                                                 

In [59]:
def writeTFRecord(filename='test.tfrecord', df=None):
    with tf.io.TFRecordWriter(filename) as writer:
        for _, row in df.iterrows():
            pass_id, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender = row.PassengerId, \
                 row.Pclass, row.Name, row.Age, row.SibSp, row.Parch, row.Ticket, row.Fare, row.Cabin, row.Embarked, row.Sex
            try:
                label = row.Survived
            except AttributeError:
                label = None

            data = serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender)
            writer.write(data)

            
        # for i in range(df):
        #     # example = serialize_example(
        #     #     feature0[i], feature1[i], feature2[i], feature3[i])
        #     # writer.write(example)
        #     pass

In [56]:
writeTFRecord('train.tfrecord', train_set)

In [52]:
raw_ds = tf.data.TFRecordDataset('train.tfrecord')

In [53]:
example = None
for raw_record in raw_ds.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

In [60]:
test_set = pd.read_csv('test.csv')
writeTFRecord('test.tfrecord', test_set)